In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import fnmatch
import cv2
import numpy as np
import string
import time
import glob
from matplotlib import pyplot as plt
# import fastwer
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.activations import relu, sigmoid, softmax
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
threshold = [32, 128]
char_list = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789\
			 ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặ\
			 ẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ "

In [ ]:
Img = np.load('/content/drive/MyDrive/Project1/Img.npy')
Padded_txt = np.load('/content/drive/MyDrive/Project1/Padded_txt.npy')
Input_length = np.load('/content/drive/MyDrive/Project1/Input_length.npy')
Label_length = np.load('/content/drive/MyDrive/Project1/Label_length.npy')
Labels = np.load('/content/drive/MyDrive/Project1/Labels.npy')
max_label_len = np.load('/content/drive/MyDrive/Project1/max_label_len.npy')

In [ ]:
from tensorflow.keras.models import Model, load_model
model = load_model("/content/drive/MyDrive/Project1/model_v4.h5")

In [ ]:
def avg_wer(wer_scores, combined_ref_len):
    return float(sum(wer_scores)) / float(combined_ref_len)


def _levenshtein_distance(ref, hyp):

    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]


def word_errors(reference, hypothesis, ignore_case=False, delimiter=' '):
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)


def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def wer(reference, hypothesis, ignore_case=False, delimiter=' '):
  
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case,
                                         delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
 
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

In [ ]:
y_true = valid_Labels
correct_char = 0
total_char = 0
correct = 0
total = 0
test_wer, test_cer = [], []
for index in range(len(valid_img)):
    prediction = model.predict(valid_img[index][None, :, :, :])
    out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                                greedy=True)[0][0])
    predicted_str = ''
    for x in (out):
        for p in x:  
            if int(p) != -1:
                predicted_str += char_list[int(p)]
    print(predicted_str)
    if predicted_str == y_true[index]:
        correct += 1
        correct_char += len(y_true[index])
    total += 1
    total_char += len(y_true[index])
    test_wer.append(wer(y_true[index], predicted_str))
    test_cer.append(cer(y_true[index], predicted_str))

avg_cer = sum(test_cer)/len(test_cer)
avg_wer = sum(test_wer)/len(test_wer)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 196ms/step
phồn
1/1 [==============================] - 0s 25ms/step
chay
1/1 [==============================] - 0s 20ms/step
cũng
1/1 [==============================] - 0s 24ms/step
họ
1/1 [==============================] - 0s 27ms/step
nhựa
1/1 [==============================] - 0s 25ms/step
chi
1/1 [==============================] - 0s 26ms/step
dì
1/1 [==============================] - 0s 25ms/step
cảm
1/1 [==============================] - 0s 24ms/step
rình
1/1 [==============================] - 0s 23ms/step
kiếm
1/1 [==============================] - 0s 171ms/step
một
1/1 [==============================] - 0s 22ms/step
cá
1/1 [==============================] - 0s 23ms/step
tạt
1/1 [==============================] - 0s 24ms/step
được
1/1 [==============================] - 0s 25ms/step
cậu
1/1 [==============================] - 0s 25ms/step
bán
1/1 [==============================] - 0s 29ms/

In [ ]:
print('Average CER: %.2f%%' % (avg_cer*100))
print('Average WER: %.2f%%' % (avg_wer*100))
print('Character Accuracy: %.2f%%' % (correct_char/total_char*100))
print('Word Accuracy: %.2f%%' % (correct/total*100))

Average CER: 6.95%
Average WER: 16.96%
Character Accuracy: 82.72%
Word Accuracy: 83.04%
